In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib notebook
# from mpl_toolkits.mplot3d import axes3d
# from ipywidgets import interact, fixed
# import ipywidgets as widgets
# from tqdm import tqdm_notebook as tqdm

# from ipywidgets import interactive, interact, HBox, Layout, VBox

In [9]:
list_of_positions = [(16,19,110, 0, 0, 0), # good, approaching high values of phi from high gamma
                     (16,25,110, 0, 0, 0), # good, approaching high values of phi from low gamma
                     (18,30, 90, 0, 0, 0), # another angle of attack
                     (20,30,110, 0, 0, 0), # this is the area where the green seems to accumulate
                     (17,19,110, 2, 2, 6), # initial cube
                     (17,25,108, 2, 2, 6), # final cube
                     (25,30,101, 0, 0, 0), # things go well
                     (25,28,99, 4, 5, 14), # this is the wavefunction I want to cut out.
                     (22,22,110, 7, 7, 20), # I want a cube that defines CI part
                    ]

folder = '/home/alessio/Desktop/d-projectExplorer/positions'
qp.ensure_dir(folder)

zero_vector = np.zeros((55,56,160))

a = 0

for x in list_of_positions:
    new = np.zeros_like(zero_vector)
    p, g, t, hms_p, hms_g, hms_t = x
    new[p-hms_p:p+hms_p+1,g-hms_g:g+hms_g+1,t-hms_t:t+hms_t+1] = 1
    dictio = {}
    dictio['Position'] = new
    dictio['label'] = '({},{},{}) ({} {} {})'.format(p, g, t, hms_p, hms_g, hms_t)
    filename = os.path.join(folder,'Position{:04}.h5'.format(a))
    a+=1
    qp.writeH5fileDict(filename, dictio)


In [7]:
def cut_out_wf(wf_name,tupleZ,outname):
    wf = qp.retrieve_hdf5_data(wf_name,'WF')
    time = qp.retrieve_hdf5_data(wf_name,'Time')
    mask = np.zeros_like(wf)
    p, g, t, hms_p, hms_g, hms_t = tupleZ
    mask[p-hms_p:p+hms_p+1,g-hms_g:g+hms_g+1,t-hms_t:t+hms_t+1] = 1
    new_wf2 = wf * mask
    new_wf = new_wf2/(np.linalg.norm(new_wf2))
    new_wf_file = {'Time': time, 'WF': new_wf}
    qp.writeH5fileDict(outname,new_wf_file)


# here I want to save IR NOR and IR INV into two different files
wf_name = '/home/alessio/w-August-Run/m-IR-Polarized-NOR_0000/Gaussian0045.h5'
outname = '/home/alessio/Desktop/d-projectExplorer/positions/Gaussian0000.h5'
    
#wf_name = '/home/alessio/w-August-Run/m-IR-Polarized-INV_0000/Gaussian0045.h5'
#outname = '/home/alessio/Desktop/d-projectExplorer/positions/Gaussian0001.h5'
tupleZ = (25,28,99, 4, 5, 14)


cut_out_wf(wf_name, tupleZ, outname)